# Setup

In [ ]:
# Importamos las librerias necesarias
import shutil
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import glob
import os
import re
from datetime import datetime
from pathlib import Path

In [ ]:
# Creamos el directorio dataset
!mkdir datasets

In [ ]:
# Esta función nos dará la lista de ficheros a mover desde Drive
def full_name_func(source_folder,dataset,file_exclusion_patterns = [],subdirectories_included = [],frames = 1):

    full_name_list = []

    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    if not file.startswith(tuple(file_exclusion_patterns)):
                        src_file = os.path.join(source_images_path, file)
                        subfolder = os.path.basename(Path(src_file).parents[1])
                        if ((subfolder in subdirectories_included) or (not subdirectories_included)):
                            full_name = dataset + '-' + subfolder +'-'+ file
                            full_name_list.append(full_name[:-4])

    full_name_list.sort()

    final_list = [file for i,file in enumerate(full_name_list) if i%frames == 0]

    return final_list

In [ ]:
# Movemos la lista de ficheros de Drive a Colab
def drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list):

    os.makedirs(destination_images, exist_ok=True)
    os.makedirs(destination_labels, exist_ok=True)

    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    src_file = os.path.join(source_images_path, file)
                    dst_file = os.path.join(destination_images, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_images, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)
            elif directory == 'labels':
                source_labels_path = os.path.join(root, directory)
                for file in os.listdir(source_labels_path):
                    src_file = os.path.join(source_labels_path, file)
                    dst_file = os.path.join(destination_labels, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_labels, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)

In [ ]:
destination_images = '/content/datasets/DELT/images'
destination_labels = '/content/datasets/DELT/labels'


dataset = 'osaconservation'
n_frames = 1

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset

# Primer test:
# s_included = ['PreArribada','102FTASK','103FTASK','105FTASK','111FTASK','116FTASK','118FTASK','121FTASK','124FTASK']
# Segundo test + test YouTube:
s_included = ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames = n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)


dataset = 'vivemar'
n_frames = 1

s_included = []

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset
file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames=n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)

In [ ]:
# Importamos YOLOv5

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt comet_ml

import torch
import utils

display = utils.notebook_init()

YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.1/235.7 GB disk)


In [ ]:
#Traemos los ficheros de configuración de Drive

!cp ../drive/MyDrive/UOC/DELT/DELT.yaml data
!cp ../drive/MyDrive/UOC/DELT/.comet.config .

In [ ]:
# Creamos la estructura de train/val/test

def split_dataset(image_names, train_size, val_size):
    for i, image_name in enumerate(image_names):

        label_name = image_name.replace('.JPG', '.txt')
        label_name = label_name.replace('.jpg', '.txt')

        if i < train_size:
            split = 'train'
        elif i < train_size + val_size:
            split = 'val'
        else:
            split = 'test'

        source_image_path = f'../datasets/DELT/images/{image_name}'
        source_label_path = f'../datasets/DELT/labels/{label_name}'

        target_image_folder = f'../datasets/DELT/images/{split}'
        target_label_folder = f'../datasets/DELT/labels/{split}'

        shutil.copy(source_image_path, target_image_folder)
        try:
          shutil.copy(source_label_path, target_label_folder) # No todas las imagenes tienen label
        except FileNotFoundError:
          pass

def get_filenames(folder):
    filenames = set()

    for path in glob.glob(os.path.join(folder, '*.*')):

        filename = os.path.split(path)[-1]
        filenames.add(filename)

    return filenames


if not os.path.exists('DELT'):
    for folder in ['images', 'labels']:
        for split in ['train', 'val', 'test']:
            os.makedirs(f'../datasets/DELT/{folder}/{split}')


turtle_images = get_filenames('../datasets/DELT/images')
turtle_images = np.array(list(turtle_images))

np.random.seed(42)
np.random.shuffle(turtle_images)

train_size = int(np.floor(len(turtle_images)*0.8))
val_size = int(np.floor(len(turtle_images)*0.2))

split_dataset(turtle_images, train_size, val_size)

# Entrenamiento

In [ ]:
#@title Select YOLOv5 🚀 model {run: 'auto'}
YOLO_model = 'yolov5s' #@param ['yolov5x','yolov5l', 'yolov5m', 'yolov5s','yolov5n']
YOLO_model_w = YOLO_model + '.pt'

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()


In [ ]:
dataset_name = 'osaconservation' + '-vivemar'

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset_name + '-' + date_string + '-' + YOLO_model + '-new_version-' + str(n_frames) +'_frames'

project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# model_v = 'bahamas-202411251318-yolov5l-new_version-1_frames'
# YOLO_model_w = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

!env COMET_LOG_PER_CLASS_METRICS=true python train.py --data data/DELT.yaml --weights {YOLO_model_w} --epochs 300 --freeze 10 --bbox_interval 1 --patience 15 --project {project_folder} --name {project_name}

Se han truncado las últimas 5000 líneas del flujo de salida.
     41/299      2.43G    0.02926    0.00714          0         45        640:  58% 105/180 [01:18<01:29,  1.19s/it]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     41/299      2.43G    0.02925   0.007139          0         42        640:  59% 106/180 [01:19<01:07,  1.09it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     41/299      2.43G    0.02927   0.007152          0         42        640:  59% 107/180 [01:21<01:32,  1.26s/it]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     41/299      2.43

In [ ]:
from google.colab import runtime
runtime.unassign()